In [1]:
import os
if os.getcwd() == '/home/user/code':
    os.chdir('/home/user/code/nlp2024_ClefTask4SOTA')

In [140]:
from TexSoup import TexSoup

def split_into_sections(tex):
    """returns: List of tuples (section_name, section_text)"""
    soup = TexSoup(tex)

    doc_text = tex

    sections = []

    sections.append(("title", soup.title.contents[0])) if soup.title else None
    sections.append(("abstract", soup.abstract.contents[0])) if soup.abstract else None
    sections.append(("tables", "\n".join([str(node) for node in soup.find_all("table")]))) if soup.find_all("table") else None


    # extract latex sections and text
    prev_section = "pre"

    for node in soup.find_all("section"):
        section_text, doc_text = doc_text.split(str(node), 1)
        sections.append((prev_section, section_text))
        prev_section = node.contents[0]
    sections.append((prev_section, doc_text))
    return sections

In [148]:
# TODO: Define Prompt function
import ollama

model = "llama3:8b"

def contains_qradruple(tex):
    return f"Decide if if the following text mentions a result on a benchmark leaderboard, including the task, dataset, metric and the score. \
    Ignore the content of the text, only output yes, if it contains a leaderboard mention or no, if no such information is included. \n \
    Text: \n {tex} \n \
    Answer with yes or no only without providing any other text. "

def zs(prompt):
    res = ollama.generate(model=model, prompt=prompt)
    context_len = len(res["context"])
    # print(f"context lenght: {context_len}")
    return res["response"]



In [165]:
def section_wise_detection(tex):
    answers = []
    try:
        sections = split_into_sections(tex)
    except:
        sections = [("full", tex)] # in the case the structure cannot be parsed:
    print(f"{len(sections)} sections")
    for section_name, section_text in sections:
        answers.append(str(zs(contains_qradruple(section_text))).lower())

    return any([answer == 'yes' for answer in answers])

   

In [172]:
# run on train data to evaluate method

from dataset import BinaryTDMSDataset, PATH

from datetime import datetime



execution_start = datetime.now().strftime("%m%d%Y-%H%M%S")

train_dataset = BinaryTDMSDataset(PATH.TRAIN)


results = []
for i in range(len(train_dataset)):
    f, tex, ground_truth = train_dataset.__getitem__(i)
    pred = section_wise_detection(tex)
    print(f"{f}: {pred} ({ground_truth})")
    results.append((f, ground_truth, pred))

import pandas as pd

df = pd.DataFrame(results)
df.columns = ["file", "ground_truth", "pred"]

df.to_feather(f"results/zs_train_{execution_start}.feather")


10 sections
context lenght: 102
context lenght: 336
context lenght: 1203
context lenght: 687
context lenght: 1450
context lenght: 1530
context lenght: 1139
context lenght: 2145
context lenght: 180
context lenght: 178
13 sections
context lenght: 93
context lenght: 174
context lenght: 1971
context lenght: 691
context lenght: 306
context lenght: 1721
context lenght: 3255
context lenght: 1238
context lenght: 2165
context lenght: 2905
context lenght: 886
context lenght: 271
context lenght: 414
1 sections
context lenght: 25778


In [ ]:
# TODO: metrics



In [ ]:
# TODO: Test set evalutation

from dataset import BinaryTDMSDataset, PATH

test_dataset = BinaryTDMSDataset(PATH.TEST)

def get_index(folder):
    return [i for i, t, j in test_dataset.all_paths].index(folder)

i, tex, jsn = test_dataset.__getitem__(get_index("0706.0014"))
